# summary

visualize model importance features and specific patients' model predictions and why.  Plot anthropometrics by visit and plot cat2/cat1 events on the timeline.

# setup

run

1.   etl
2.   etl_derioration

to get the dataframes that this notebook depends on



In [1]:
# TODO change to digital ocean
MODEL_PATH = "/content/drive/My Drive/[PBA] Code/model"
# above which percentile should shap values be computed (as it's resource intensive)
TOP_PCT = 0.50
# export SHAP waterfall plot values (adds to runtime)
EXPORT_SHAP_WATERFALL = True

In [2]:

!pip install import_ipynb --quiet

# shapley explanation of models
!pip install shap --quiet
# survival analysis
!pip install lifelines --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 7.6 MB/s eta 0:00:00


In [ ]:
!git clone -b brian-etl-code https://github.com/The-Taimaka-Project/health-predictions.git
#!git clone https://github.com/The-Taimaka-Project/health-predictions.git

# Change directory to the repository
%cd health-predictions/taimaka_health_predictions/inference
%pwd
#TODO move util here

%cd /content

Cloning into 'health-predictions'...
remote: Enumerating objects: 305, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 305 (delta 79), reused 57 (delta 24), pack-reused 136 (from 1)
Receiving objects: 100% (305/305), 8.48 MiB | 8.74 MiB/s, done.
Resolving deltas: 100% (110/110), done.
/content/health-predictions/packages/inference/run
/content


In [4]:
import pandas as pd

# Set global output format to Pandas
from sklearn import set_config
set_config(transform_output="pandas")

# Set global output format to Pandas
import datetime
import numpy as np
import pickle
import os
import re
import json

#import shap
import import_ipynb
from warnings import simplefilter,filterwarnings

from lifelines import WeibullAFTFitter



from google.colab import drive
drive.mount('/content/drive')


os.chdir("/content/drive/My Drive/[PBA] Code")

#from util import days_since_min,make_populated_column,logistic_regression,AutogluonWrapper,explain_prediction,plot_anthros,check_cols,plot_corr,drop_result_columns
#from util import decision_tree,print_tree,explain_tree_sample,print_patient_probabilities,impute_missing_values,select_logistic_features,get_best_lin_model,explain_logreg
#from util import get_best_tree_model,decision_tree_f1,logistic_train,plot_corr_jitter,lightgbm_train,select_features,make_dummy_columns,split_detn_new_onset_medical_complication,ag_model_load
#from util import label_test,make_test,label_test_tree,label_test_lin,plot_survival,plot_survival3
#from util import ag_feature_generator,label_test_gbm,explain_prediction_gbm,log_odds_to_probability,merge_probabilities,ag_regress_model_load,ag_model_load_suffix
from util import reduce_dimensionality,split_detn_new_onset_medical_complication



os.chdir("/content")

# prompt: read google shared drive file


dir = "/content/drive/My Drive/[PBA] Data/analysis/"
MODEL_PATH = "/content/drive/My Drive/[PBA] Code/model"



Mounted at /content/drive


In [5]:
# use auto ML (autogluon) to predict the 5 deterioration events
!pip install autogluon --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 8.3 MB/s eta 0:00

In [6]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator
from autogluon.tabular import TabularDataset, TabularPredictor


In [7]:
import logging
# Create a logger# Create a logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG) # Set the minimum logging level

# Create a handler to output logs to the console
console_handler = logging.StreamHandler()

file_handler = logging.FileHandler('my_log.log')
file_handler.setLevel(logging.INFO) # Set the logging level for the handler


# Create a formatter to specify the log message format
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(lineno)d - %(funcName)s - %(message)s')

# Add the formatter to the handler

file_handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(file_handler)


In [8]:
# TODO replace with csv
with open(dir + 'admit_weekly.pkl', 'rb') as f:
  admit_weekly = pickle.load(f)
with open(dir + 'admit_current.pkl', 'rb') as f:
  admit_current = pickle.load(f)

In [9]:
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=FutureWarning)

In [10]:
pid_probabilities = admit_current[['pid','status','site_current']].copy()
active_pids = pid_probabilities[pid_probabilities['status']=='active']['pid'].unique()
logger.debug(pid_probabilities.shape)

DEBUG:my_logger:(10322, 3)


In [11]:
def read_detn(label):
  with open(dir + f'{label}.pkl', 'rb') as f:
    detn = pickle.load(f)
  return detn


# new onset medical complication

New onset medical complication - 'cat1' complication (see vars in raw ODK data with cat1_ prefix)

In [12]:
# prompt: use pickle to read deterioration dataframe
label = 'new_onset_medical_complication'

#detn = read_detn(label)

with open(dir + f'{label}.pkl', 'rb') as f:
  detn = pickle.load(f)
logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
# (5215, 1786) 512 0.09817833173537871 version 1, 11/15
# (5215, 1831) 524 0.10047938638542665 version 2
# (5215, 1615)
#detn = drop_result_columns(detn,label)
#detn.drop(columns=[f'{label}_date'],inplace=True)
detn = detn[detn['weekly_last_admit'].notnull()].copy()

detn.drop(columns='c_admission_other',inplace=True) # causes duplicate column if get_dummies invoked
detn.drop(columns='phone_owner_other',inplace=True) # messes up get_dummies
detn.drop(columns='calc_dayssincevita',inplace=True) # incorrectly calculated for non-active, messes up detn_wk1_only models

y_cat1_cols = [col for col in detn.columns if col.startswith('y_cat1')]
detn.drop(columns=y_cat1_cols,inplace=True) # only looking at label new_onset_medical_complication, not the actual categories

logger.debug(detn.shape)

#detn['glbsite'] = detn['glbsite'].str.lower()
#detn['wk1_autosite'] = detn['wk1_autosite'].str.lower()
detn['wk1_calc_los'] = round(detn['wk1_calc_los'],0)
detn['weekly_last_muac'] = round(detn['weekly_last_muac'],1)

detn['wk1_calc_los'].fillna(0,inplace=True)
LOS_CUTOFF = 12
MUAC_CUTOFF = 12.5
logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()},{detn.shape}')


detn = detn[(detn['weekly_last_muac']< MUAC_CUTOFF) & (detn['wk1_calc_los']< LOS_CUTOFF) ].copy()

detn = reduce_dimensionality(detn,['muac_diff_ratio','muac'],'muac_diff_ratio_z')
detn = reduce_dimensionality(detn,['household_adults','household_slept','living_children'],'household_adults_slept_living_children_z')

logger.debug(f'{detn[label].sum()},{detn[label].mean()},{detn.shape}')


DEBUG:my_logger:((10322, 2345), np.int64(1136)),0.11005619066072467
DEBUG:my_logger:(9747, 2328)
DEBUG:my_logger:((9747, 2328), np.int64(1136)),0.11654868164563456,(9747, 2328)
DEBUG:my_logger:1051,0.47087813620071683,(2232, 2330)


## predictive model importances

### AutoGluon

#### stratified models by length of service

In [13]:
def run_ag_model(label,detn,suffix):
  from autogluon.tabular import TabularDataset, TabularPredictor
  from util import AutogluonWrapper
  import shap


  model_path = f"{MODEL_PATH}/{label}{suffix}/"
  predictor = TabularPredictor.load(model_path,require_py_version_match=False,require_version_match=False)

  ag_features = predictor.features()
  logger.debug(f'ag features:, {len(ag_features)}, {ag_features}')


  y_pred_proba_all = predictor.predict_proba(detn[ag_features])

  target_class = 1  # can be any possible value of the label column
  negative_class = 0
  baseline = detn[ag_features][detn[label]==negative_class].sample(20, random_state=0)
  ag_wrapper = AutogluonWrapper(predictor, ag_features, target_class)
  explainer = shap.KernelExplainer(ag_wrapper.predict_proba, baseline)


  return y_pred_proba_all,explainer,ag_features

In [14]:
def export_waterfall_shap_values(explainer2,detn,features):
  import shap

  shap_values_all = explainer2.shap_values(detn[features]) # Calculate SHAP values

  exp_all = shap.Explanation(shap_values_all,
                       explainer2.expected_value,
                       data=detn[features].values,
                       feature_names=features)


  # prompt: create a json object combining each array occurrence of exp_all.values with the corresponding array occurence of exp_all.data

  json_objects = []
  for shap_values_array, feature_values_array in zip(exp_all.values, exp_all.data):
      # Create a dictionary for the current instance
      data_dict = {
          'base_probability': exp_all.base_values,
          'shap_values': shap_values_array.tolist(),
          'feature_values': feature_values_array.tolist(),
          'feature_names': exp_all.feature_names
      }
      json_objects.append(data_dict)

  # prompt: zip detn['pid'] and json_objects into a dict keyed by detn['pid']

  json_dict = dict(zip(detn['pid'], json_objects))


  # Example: Create a Series from the json_dict where the index is the pid
  json_series = pd.Series(json_dict)
  return json_series



In [15]:
detn_admit_only,_,_,_ = split_detn_new_onset_medical_complication(detn,label)

#detn_admit_only = make_dummy_columns(detn_admit_only)
pid_not_in_admit = detn[~detn['pid'].isin(detn_admit_only['pid'])]['pid']
detn_filtered = detn[detn['pid'].isin(pid_not_in_admit)].copy()



(433, 536) (424, 1436) (421, 1883) (954, 2330)


In [16]:
y_pred_proba_all1,explainer1,ag_features1 = run_ag_model(label,detn_admit_only,'1')
y_pred_proba_all2,explainer2,ag_features2 = run_ag_model(label,detn_filtered,'not1')


Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.12, current=3.11.13)
DEBUG:my_logger:ag features:, 7, ['admit_cat1_complications', 'estdist_home', 'household_adults_slept_living_children_z', 'age_takefamily', 'rainy_season_admit', 'temperature', 'resp_rate']
Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.12, current=3.11.13)
DEBUG:my_logger:ag features:, 14, ['hl_trend', 'weekly_avg_muac', 'wk2_muac', 'estdist_home', 'wk1_rainy_season_weekly', 'resp_rate', 'muac_trend', 'cat1_complications_weekly', 'household_adults_slept_living_children_z', 'muac_diff_ratio_z', 'muac_rsquared', 'muac_diff_ratio_rate', 'wk1_weight_diff_rate', 'wk1_resp_rate_weekly']


In [17]:
#y_pred_proba_all_stratified = pd.concat([y_pred_proba_all1,y_pred_proba_all2,y_pred_proba_all3,y_pred_proba_all4],axis=0)
y_pred_proba_all_stratified = pd.concat([y_pred_proba_all1,y_pred_proba_all2],axis=0)
y_pred_proba_all_stratified_series = y_pred_proba_all_stratified[1].rename(f'probability_{label}_stratified')
pid_probabilities = pid_probabilities.join(y_pred_proba_all_stratified_series)

In [18]:
pid_probabilities[f'percentrank_{label}_stratified'] = pid_probabilities[f'probability_{label}_stratified'].rank(pct=True)
top_pct_pids = pid_probabilities[pid_probabilities[f'percentrank_{label}_stratified'] > TOP_PCT]['pid'].unique()


In [19]:
if EXPORT_SHAP_WATERFALL:
  json_series = export_waterfall_shap_values(explainer1,detn_admit_only[(detn_admit_only['pid'].isin(active_pids)) & (detn_admit_only['pid'].isin(top_pct_pids)) & (detn_admit_only[label]== 0)],ag_features1)
  json_series2 = export_waterfall_shap_values(explainer2,detn_filtered[(detn_filtered['pid'].isin(active_pids)) & (detn_filtered['pid'].isin(top_pct_pids)) & (detn_filtered[label]== 0)],ag_features2)
  pid_probabilities = pd.merge(pid_probabilities, pd.concat([json_series, json_series2]).rename(f'{label}_shap_data'), left_on='pid', right_index=True, how='left')


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

#### all los in single model

### Light Gradient-Boosting Machine (GBM) classifier

### explain via Decision Tree Classifier

### explain via logistic linear regression

## survival

In [20]:
# survival
logger.debug(f'{detn[label].sum()},{detn.shape}')
detn = pd.merge(detn, admit_current[['pid','status','status_date']], on='pid', how='inner')
logger.debug(f'{detn[label].sum()},{detn.shape}')

label_date = f'{label}_date'

detn.loc[detn[label] == 0, label_date] = None

# prompt: filter detn for label_date not null or wk1_calcdate_weekly not null

# drop those w/o a visit, hence no lifetime to be plotted)
detn = detn[(detn[label_date].notnull()) | (detn['wk1_calcdate_weekly'].notnull())].copy()
logger.debug(f'{detn[label].sum()},{detn.shape}')



DEBUG:my_logger:1051,(2232, 2330)
DEBUG:my_logger:1051,(2232, 2332)
DEBUG:my_logger:1051,(2063, 2332)


In [21]:
# prompt: create column in detn called final_date which is status_date or wk1_calcdate_weekly whichever is greater
# Create 'final_date' column, choosing the later date between 'status_date' and 'wk1_calcdate_weekly'
detn['final_date'] = detn[['status_date', 'wk1_calcdate_weekly']].max(axis=1)

# Fill NaN (active patients w/o a visit) values in 'final_date' with today's date
detn['final_date'] = detn['final_date'].fillna(datetime.date.today())
detn['final_date'] = pd.to_datetime(detn['final_date'])
detn['duration_days'] = detn['final_date'] - detn['calcdate']
detn['duration_days'] = detn['duration_days'].dt.days

logger.debug(detn[label_date].notnull().sum())


DEBUG:my_logger:1051


In [22]:
# prompt: for rows in detn where label==1 set duration_days to label_date - calcdate

# Calculate duration_days only for rows where label == 1
detn.loc[detn[label] == 1, 'duration_days'] = (detn.loc[detn[label] == 1, label_date] - detn.loc[detn[label] == 1, 'calcdate']).dt.days


In [23]:
# prompt: clip detn['duration_days'] to 0.1

detn['duration_days'] = detn['duration_days'].clip(lower=0.1)


### AFT survival regression

In [24]:
# AFT survival regression
selected_columns = ['muac_diff','wk1_muac','muac_diff_ratio_rate']

X = detn[selected_columns].copy()

data_y = detn[[label,'duration_days']].copy()
data_x_numeric = X.copy()
data_y = detn[[label,'duration_days']].copy()

regression_dataset=data_x_numeric.join(data_y)

In [25]:
regr_cols = ['wk1_muac','muac','muac_diff','muac_diff_ratio_rate',label,'duration_days','pid','status']
regression_dataset = detn[regr_cols].copy()
regression_dataset['wk1_muac'].fillna(regression_dataset['muac'],inplace=True)
regression_dataset.drop(columns=['muac'],inplace=True)
regression_dataset.dropna(inplace=True)
#regression_dataset['wk1_muac'].fillna(0,inplace=True)


regression_dataset['duration_days'] = regression_dataset['duration_days'].clip(lower=.01)
aft = WeibullAFTFitter()

aft.fit(regression_dataset, duration_col='duration_days', event_col=f'{label}', ancillary=True, formula=' wk1_muac + muac_diff + muac_diff_ratio_rate')

logger.debug(aft.print_summary())



<lifelines.WeibullAFTFitter: fitted with 1799 total observations, 1012 right-censored observations>
             duration col = 'duration_days'
                event col = 'new_onset_medical_complication'
   number of observations = 1799
number of events observed = 787
           log-likelihood = -3585.08
         time fit was run = 2025-06-14 22:54:20 UTC

---
                               coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                              
lambda_ Intercept              4.21     67.02      0.20            3.81            4.60               44.96               99.90
        wk1_muac              -0.04      0.96      0.02           -0.07           -0.00                0.93                1.00
        muac_diff              0.52      1.69      0.02            0.48            0.57                1.61                1.77
        muac_diff_ratio_rate -66.79      0.00      2.99          -72.64          -60.94                0.00                0.00
rho_    Intercept             -0.84      0.43      0.29           -1.40           -0.27                0.25                0.76
        wk1_muac               0.14      1.15      0.03            0.09            0.19                1.09                1.21
        muac_diff              0.36      1.43      0.04            0.27            0.44                1.31                1.56
        muac_diff_ratio_rate -16.04      0.00      4.10          -24.08           -8.01                0.00                0.00

                              cmp to      z      p  -log2(p)
param   covariate                                           
lambda_ Intercept               0.00  20.65 <0.005    312.23
        wk1_muac                0.00  -2.01   0.04      4.50
        muac_diff               0.00  21.35 <0.005    333.43
        muac_diff_ratio_rate    0.00 -22.37 <0.005    365.90
rho_    Intercept               0.00  -2.91 <0.005      8.11
        wk1_muac                0.00   5.41 <0.005     23.90
        muac_diff               0.00   8.06 <0.005     50.23
        muac_diff_ratio_rate    0.00  -3.91 <0.005     13.42
---
Concordance = 0.82
AIC = 7186.15
log-likelihood ratio test = 643.29 on 6 df
-log2(p) of ll-ratio test = 448.37

DEBUG:my_logger:None


column	description
1. wk1_muac	muac on most recent visit
1. muac_diff_ratio_rate	latest/last muac minus first muac divided by first muac divided by the number of days in the program.  So muac change per starting muac per day.  Last would the last visit or the last visit up to the appropriate deterioration type.
1. muac_diff	the difference between the most recent (before deterioration) muac and the first visit's muac


### censored (i.e., patients w/o complication yet) survival prediction

In [26]:
# censored (i.e., patients w/o complication yet) survival prediction

regr_cols = ['wk1_muac','muac','muac_diff','muac_diff_ratio_rate',label,'duration_days','pid','status']
censored_subjects = detn[regr_cols].copy()
censored_subjects['wk1_muac'].fillna(censored_subjects['muac'],inplace=True)
censored_subjects.drop(columns=['muac'],inplace=True)

regression_dataset[regression_dataset['wk1_muac'].notnull()][['muac_diff','muac_diff_ratio_rate']].isnull().sum()

censored_subjects['muac_diff_ratio_rate'].fillna(0,inplace=True)

censored_subjects.dropna(inplace=True)

censored_subjects = censored_subjects.loc[detn[label]==0].copy()

duration_days_col = f'duration_days_{label}'

censored_subjects.rename(columns={'duration_days':duration_days_col},inplace=True)

censored_subjects_last_obs = censored_subjects[duration_days_col]


y_median = aft.predict_median(censored_subjects,conditional_after=censored_subjects_last_obs)
y_median.rename(f'median_days_to_{label}',inplace=True)

df = pd.merge(censored_subjects[['pid','wk1_muac','muac_diff','muac_diff_ratio_rate',duration_days_col]],y_median,left_index=True,right_index=True)

pid_probabilities =pd.merge(pid_probabilities,df,on='pid',how='left')




## visualize most and least probable patients

### most probable

### 2nd most probable

# poor weight gain

* Weight at week 3 is lower than weight at admission
* Weight loss for 3 consecutive weeks (not related to loss of oedema)
* Static weight or weight loss for 4 consecutive weeks
* Poor weight gain (<5 g/kg/day) for 4 consecutive weeks


The biggest predictors of poor weight gain are ['wk1_muac','wk2_muac','weight_trend','wfh_trend']

1.   Most recent visit's MUAC
2.   2nd most recent visit's MUAC
3. Weight trend
4. Weight for height trend



In [27]:
# prompt: use pickle to get the data to train and infer on

label = 'detn_weight_loss_ever'

#with open(dir + f'{label}.pkl', 'rb') as f:
#  detn = pickle.load(f)
detn = read_detn(label)

logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
# (5215, 1786) 1021 0.19578139980824544 for 70% training phase
# (8352, 1844) 1660 0.19875478927203066
detn['wk1_calc_los'].fillna(0,inplace=True)

LOS_CUTOFF = 12
MUAC_CUTOFF = 12.7

detn = detn[((detn['wk1_b_discharged']==0) & (detn['weekly_last_muac']< MUAC_CUTOFF) & (detn['wk1_calc_los']< LOS_CUTOFF)) ].copy()


logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
detn = reduce_dimensionality(detn,['household_adults','household_slept','living_children'],'household_adults_slept_living_children_z')
detn = reduce_dimensionality(detn,['resp_rate', 'temperature'],'resp_rate_temperature')
#detn['phq_score'] = detn['phq_score'].astype(int)

logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')


<ipython-input-11-4214077459>:3: DtypeWarning: Columns (4,5,6,9,27,40,75,77,78,89,91,96,101,103,123,124,135,136,137,138,141,143,144,145,153,159,161,162,164,165,166,167,177,179,224,244,246,247,275,284,288,296,303,309,315,322,324,325,328,335,336,337,338,339,340,342,343,401,411,442,443,444,445,452,475,477,482,487,489,512,521,546,548,568,748,749,750,755,757,871,880,900,909,919,940,941,942,944,949,1012,1013,1015,1016,1017,1018,1019,1030,1031,1035,1036,1037,1042,1043,1066,1084,1085,1086,1087,1089,1092,1093,1094,1099,1102,1103,1104,1105,1108,1111,1117,1118,1120,1121,1124,1126,1127,1128,1130,1132,1133,1134,1135,1140,1160,1164,1165,1168,1199,1213,1214,1216,1218,1219,1220,1221,1222,1227,1228,1231,1232,1233,1234,1235,1238,1240,1241,1242,1244,1245,1246,1247,1248,1254,1255,1256,1257,1258,1259,1261,1262,1265,1266,1267,1268,1269,1270,1271,1272,1273,1274,1275,1276,1278,1279,1280,1282,1286,1287,1288,1289,1290,1291,1292,1293,1294,1295,1302,1303,1305,1306,1312,1313,1314,1316,1317,1318,1319,1320,1321,1322

## predictive model feature importances

### autogluon

90% of its predictions are right (precision).  It found 87% of the poor weight gain patients (recall).



In [28]:
# infer
y_pred_proba_all,explainer,ag_features = run_ag_model(label,detn,'')

Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.12, current=3.11.13)
DEBUG:my_logger:ag features:, 8, ['wk1_weight_diff_rate', 'wk1_calc_los', 'hl_trend', 'hfa_trend', 'wfa_trend', 'weekly_last_muac', 'household_adults_slept_living_children_z', 'weekly_last_wfh']


In [29]:
pred_proba_all_series = y_pred_proba_all[1].rename(f'probability_{label}')
pid_probabilities = pid_probabilities.join(pred_proba_all_series)
logger.debug(pid_probabilities.shape)

DEBUG:my_logger:(10322, 12)


In [30]:
pid_probabilities[f'percentrank_{label}'] = pid_probabilities[f'probability_{label}'].rank(pct=True)
top_pct_pids = pid_probabilities[pid_probabilities[f'percentrank_{label}'] > TOP_PCT]['pid'].unique()
if EXPORT_SHAP_WATERFALL:
  json_series = export_waterfall_shap_values(explainer,detn[(detn['pid'].isin(active_pids)) & (detn['pid'].isin(top_pct_pids)) & (detn[label]== 0)],ag_features)
  pid_probabilities = pd.merge(pid_probabilities, json_series.rename(f'{label}_shap_data'), left_on='pid', right_index=True, how='left')


  0%|          | 0/7 [00:00<?, ?it/s]

## survival

In [31]:
# survival
logger.debug(f'{detn[label].sum()},{detn.shape}')
detn = pd.merge(detn, admit_current[['pid','status','status_date']], on='pid', how='inner')
logger.debug(f'{detn[label].sum()},{detn.shape}')

label_date = f'{label}_date'

detn.loc[detn[label] == 0, label_date] = None

# prompt: filter detn for label_date not null or wk1_calcdate_weekly not null

# drop those w/o a visit, hence no lifetime to be plotted)
detn = detn[(detn[label_date].notnull()) | (detn['wk1_calcdate_weekly'].notnull())].copy()
logger.debug(f'{detn[label].sum()},{detn.shape}')



DEBUG:my_logger:2001,(3303, 2333)
DEBUG:my_logger:2001,(3303, 2335)
DEBUG:my_logger:2001,(2475, 2335)


In [32]:
# prompt: create column in detn called final_date which is status_date or wk1_calcdate_weekly whichever is greater
# Create 'final_date' column, choosing the later date between 'status_date' and 'wk1_calcdate_weekly'
detn['final_date'] = detn[['status_date', 'wk1_calcdate_weekly']].max(axis=1)

# Fill NaN (active patients w/o a visit) values in 'final_date' with today's date
detn['final_date'] = detn['final_date'].fillna(datetime.date.today())
detn['final_date'] = pd.to_datetime(detn['final_date'])
detn['duration_days'] = detn['final_date'] - detn['calcdate']
detn['duration_days'] = detn['duration_days'].dt.days

logger.debug(detn[label_date].notnull().sum())


DEBUG:my_logger:2001


In [33]:
# prompt: for rows in detn where label==1 set duration_days to label_date - calcdate

# Calculate duration_days only for rows where label == 1
detn.loc[detn[label] == 1, 'duration_days'] = (detn.loc[detn[label] == 1, label_date] - detn.loc[detn[label] == 1, 'calcdate']).dt.days


### AFT survival regression

In [34]:
selected_columns = ['weight_trend','pid']


X = detn[selected_columns].copy()

data_y = detn[[label,'duration_days']].copy()
data_x_numeric = X.copy()
data_y = detn[[label,'duration_days']].copy()

regression_dataset=data_x_numeric.join(data_y)

regression_dataset = regression_dataset.dropna()

In [35]:
aft = WeibullAFTFitter()

aft.fit(regression_dataset, duration_col='duration_days', event_col=f'{label}', ancillary=True,formula='weight_trend')
logger.debug(aft.print_summary())


<lifelines.WeibullAFTFitter: fitted with 2428 total observations, 467 right-censored observations>
             duration col = 'duration_days'
                event col = 'detn_weight_loss_ever'
   number of observations = 2428
number of events observed = 1961
           log-likelihood = -8644.44
         time fit was run = 2025-06-14 22:54:41 UTC

---
                      coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                     
lambda_ Intercept     3.85     47.08      0.01            3.83            3.87               46.03               48.14
        weight_trend  6.50    664.77      0.37            5.78            7.22              323.00             1368.18
rho_    Intercept     0.91      2.48      0.02            0.87            0.94                2.40                2.56
        weight_trend  2.64     14.05      0.49            1.69            3.60                5.40               36.57

                      cmp to      z      p  -log2(p)
param   covariate                                   
lambda_ Intercept       0.00 337.83 <0.005       inf
        weight_trend    0.00  17.65 <0.005    229.16
rho_    Intercept       0.00  52.77 <0.005       inf
        weight_trend    0.00   5.41 <0.005     23.95
---
Concordance = 0.64
AIC = 17296.88
log-likelihood ratio test = 334.85 on 2 df
-log2(p) of ll-ratio test = 241.54

DEBUG:my_logger:None


### censored (i.e., patients w/o weight loss yet) survival prediction

In [36]:
censored_subjects=data_x_numeric.join(data_y)

censored_subjects['weight_trend'].fillna(0,inplace=True)

censored_subjects = censored_subjects.loc[censored_subjects[label]==0].copy()

duration_days_col = f'duration_days_{label}'

censored_subjects.rename(columns={'duration_days':duration_days_col},inplace=True)

censored_subjects_last_obs = censored_subjects[duration_days_col]

y_median = aft.predict_median(censored_subjects,conditional_after=censored_subjects_last_obs)
y_median.rename(f'median_days_to_{label}',inplace=True)

df = pd.merge(censored_subjects[['pid','weight_trend',duration_days_col]],y_median,left_index=True,right_index=True)

pid_probabilities =pd.merge(pid_probabilities,df,on='pid',how='left')
logger.debug(pid_probabilities.shape)


DEBUG:my_logger:(10322, 17)


## visualize most and least probable patients

### most probable

# muac loss 2 weeks consecutive

Static MUAC or MUAC loss for 2 consecutive weeks

if malnourish status is coded non-healthy (mam in particular) on most recent visit, probability of MUAC loss in the following 2 weeks increases 5x from the baseline 10% to 50%.

For the 2.6% with healthy MUAC but cat2_overall diagnosis on their most recent visit their probability increases 20x (from the baseline for healthy MUAC) to 37%. For the 10% with the difference between visit 1 and visit 2 MUAC is less than .5 cm and they have unhealthy MUAC, then their probability increases another 25 points (from the baseline for unhealthy MUAC), to 81%.

In [37]:
# prompt: use pickle to read deterioration dataframe
label = 'muac_loss_2_weeks_consecutive'

with open(dir + f'{label}.pkl', 'rb') as f:
  detn = pickle.load(f)

#detn = read_detn(label)


logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
# (5215, 1786) 469 0.08993288590604027 first version of data
# (8352, 1844) 817 0.09782088122605365

DEBUG:my_logger:((10322, 2331), np.int64(1229)),0.11906607246657624


In [38]:
detn['wk1_calc_los'].fillna(0,inplace=True)

LOS_CUTOFF = 12
MUAC_CUTOFF = 12.7
# prompt: detn where wk1_muac between 11.5 and 12.6 and wk1_calc_los between 10 and 12
#detn = detn[(detn['weekly_last_muac']< MUAC_CUTOFF) & (detn['wk1_calc_los']< LOS_CUTOFF) ].copy()

detn = detn[((detn['wk1_b_discharged']==0) & (detn['weekly_last_muac']< MUAC_CUTOFF) & (detn['wk1_calc_los']< LOS_CUTOFF)) ].copy()


logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
detn = reduce_dimensionality(detn,['household_adults','household_slept','living_children'],'household_adults_slept_living_children_z')



detn = reduce_dimensionality(detn,['weekly_avg_muac','weekly_last_wfh',],'muac_wfh_z')

detn = reduce_dimensionality(detn,['wk1_muac_diff_rate','muac_diff_ratio_rate','muac_diff_ratio'],'muac_diff_rate_ratio_z')


#detn['phq_score'] = detn['phq_score'].astype(int)

logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')

DEBUG:my_logger:((2485, 2331), np.int64(1148)),0.4619718309859155
DEBUG:my_logger:((2485, 2334), np.int64(1148)),0.4619718309859155


## correlations

## predictive model feature importances

### Autogluon

In [39]:
# run inference
y_pred_proba_all,explainer,ag_features = run_ag_model(label,detn,'')

Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.12, current=3.11.13)
DEBUG:my_logger:ag features:, 13, ['wk2_muac_diff_rate', 'wk1_weight_diff_rate', 'muac_rsquared', 'hfa_trend', 'wfa_trend', 'wk1_muac', 'muac_diff_rate_ratio_z', 'resp_rate', 'hl_trend', 'household_adults_slept_living_children_z', 'wk1_calc_los', 'muac_trend', 'wk2_muac']


In [40]:
pred_proba_all_series = y_pred_proba_all[1].rename(f'probability_{label}')
pid_probabilities = pid_probabilities.join(pred_proba_all_series)
logger.debug(pid_probabilities.shape)

DEBUG:my_logger:(10322, 18)


In [41]:
pid_probabilities[f'percentrank_{label}'] = pid_probabilities[f'probability_{label}'].rank(pct=True)
top_pct_pids = pid_probabilities[pid_probabilities[f'percentrank_{label}'] > TOP_PCT]['pid'].unique()
if EXPORT_SHAP_WATERFALL:
  json_series = export_waterfall_shap_values(explainer,detn[(detn['pid'].isin(active_pids)) & (detn['pid'].isin(top_pct_pids)) & (detn[label]== 0)],ag_features)
  pid_probabilities = pd.merge(pid_probabilities, json_series.rename(f'{label}_shap_data'), left_on='pid', right_index=True, how='left')


  0%|          | 0/62 [00:00<?, ?it/s]

## survival

In [42]:
# survival
logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
detn = pd.merge(detn, admit_current[['pid','status','status_date']], on='pid', how='inner')
logger.debug(f'{detn.shape,detn[label].sum()}')

label_date = f'{label}_date'

detn.loc[detn[label] == 0, label_date] = None

# prompt: filter detn for label_date not null or wk1_calcdate_weekly not null

# drop those w/o a visit, hence no lifetime to be plotted)
detn = detn[(detn[label_date].notnull()) | (detn['wk1_calcdate_weekly'].notnull())].copy()
logger.debug(f'{detn.shape,detn[label].sum()}')



DEBUG:my_logger:((2485, 2334), np.int64(1148)),0.4619718309859155
DEBUG:my_logger:((2485, 2336), np.int64(1148))
DEBUG:my_logger:((2127, 2336), np.int64(1148))


In [43]:
# prompt: create column in detn called final_date which is status_date or wk1_calcdate_weekly whichever is greater
# Create 'final_date' column, choosing the later date between 'status_date' and 'wk1_calcdate_weekly'
detn['final_date'] = detn[['status_date', 'wk1_calcdate_weekly']].max(axis=1)

# Fill NaN (active patients w/o a visit) values in 'final_date' with today's date
detn['final_date'] = detn['final_date'].fillna(datetime.date.today())
detn['final_date'] = pd.to_datetime(detn['final_date'])
detn['duration_days'] = detn['final_date'] - detn['calcdate']
detn['duration_days'] = detn['duration_days'].dt.days

logger.debug(f'{detn.shape,detn[label].sum()}')


DEBUG:my_logger:((2127, 2338), np.int64(1148))


In [44]:
# prompt: for rows in detn where label==1 set duration_days to label_date - calcdate

# Calculate duration_days only for rows where label == 1
detn.loc[detn[label] == 1, 'duration_days'] = (detn.loc[detn[label] == 1, label_date] - detn.loc[detn[label] == 1, 'calcdate']).dt.days

detn['duration_days'] = detn['duration_days'].clip(lower=0.1)



### Hazard function (rate of deterioration)

### AFT survival regression

In [45]:
selected_columns = ['wfh_trend','pid']

#print(detn[selected_columns].dtypes)

X = detn[selected_columns].copy()

data_y = detn[[label,'duration_days']].copy()
data_x_numeric = X.copy()
data_y = detn[[label,'duration_days']].copy()

regression_dataset=data_x_numeric.join(data_y)

regression_dataset = regression_dataset.dropna()

In [46]:
aft = WeibullAFTFitter()
aft.fit(regression_dataset, duration_col='duration_days', event_col=f'{label}', ancillary=True,formula='wfh_trend')
logger.debug(aft.print_summary())


<lifelines.WeibullAFTFitter: fitted with 2104 total observations, 972 right-censored observations>
             duration col = 'duration_days'
                event col = 'muac_loss_2_weeks_consecutive'
   number of observations = 2104
number of events observed = 1132
           log-likelihood = -5373.40
         time fit was run = 2025-06-14 23:01:33 UTC

---
                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                   
lambda_ Intercept   4.06     57.84      0.01            4.03            4.09               56.18               59.55
        wfh_trend 224.52  3.22e+97     35.54          154.87          294.17            1.82e+67           5.70e+127
rho_    Intercept   0.86      2.37      0.02            0.82            0.90                2.27                2.47
        wfh_trend  63.28  3.04e+27     20.15           23.78          102.78            2.14e+10            4.33e+44

                   cmp to      z      p  -log2(p)
param   covariate                                
lambda_ Intercept    0.00 273.22 <0.005       inf
        wfh_trend    0.00   6.32 <0.005     31.82
rho_    Intercept    0.00  40.29 <0.005       inf
        wfh_trend    0.00   3.14 <0.005      9.21
---
Concordance = 0.59
AIC = 10754.80
log-likelihood ratio test = 54.48 on 2 df
-log2(p) of ll-ratio test = 39.30

DEBUG:my_logger:None


### censored (i.e., patients w/o consecutive muac loss yet) survival prediction

In [47]:
censored_subjects=data_x_numeric.join(data_y)

censored_subjects['wfh_trend'].fillna(0,inplace=True)

censored_subjects = censored_subjects.loc[censored_subjects[label]==0].copy()

duration_days_col = f'duration_days_{label}'

censored_subjects.rename(columns={'duration_days':duration_days_col},inplace=True)

censored_subjects_last_obs = censored_subjects[duration_days_col]

y_median = aft.predict_median(censored_subjects,conditional_after=censored_subjects_last_obs)
y_median.rename(f'median_days_to_{label}',inplace=True)

df = pd.merge(censored_subjects[['pid','wfh_trend',duration_days_col]],y_median,left_index=True,right_index=True)

pid_probabilities =pd.merge(pid_probabilities,df,on='pid',how='left')
logger.debug(pid_probabilities.shape)




DEBUG:my_logger:(10322, 23)


## visualize most and least probable patients

### most probable

# nonresponse

(status == ‘nonresponse’)
discharge as not responding to treatment

In [48]:
# read deterioration file
label = 'nonresponse'

#detn = read_detn(label)
with open(dir + f'{label}.pkl', 'rb') as f:
  detn = pickle.load(f)


logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
# (8352, 1844) 403 0.04825191570881226


DEBUG:my_logger:((10322, 2331), np.int64(584)),0.056578182522766904


In [49]:
# prompt: create column in detn called final_date which is status_date or wk1_calcdate_weekly whichever is greater
detn = pd.merge(detn, admit_current[['pid','status','status_date']], on='pid', how='inner')
logger.debug(f'{detn[label].sum()},{detn.shape}')


# Create 'final_date' column, choosing the later date between 'status_date' and 'wk1_calcdate_weekly'
detn['final_date'] = detn[['status_date', 'wk1_calcdate_weekly']].max(axis=1)

# Fill NaN (active patients w/o a visit) values in 'final_date' with today's date
detn['final_date'] = detn['final_date'].fillna(datetime.date.today())
detn['final_date'] = pd.to_datetime(detn['final_date'])
detn['duration_days'] = detn['final_date'] - detn['calcdate']
detn['duration_days'] = detn['duration_days'].dt.days

# prompt: only keep detn rows where duration between label==1 min(duration_days) and max(duration_days)

# Assuming 'detn' DataFrame is already loaded as in the provided code.

min_duration = detn.groupby(label)['duration_days'].min()[1]
max_duration = detn.groupby(label)['duration_days'].max()[1]

detn_filtered = detn[
    (detn['duration_days'] >= min_duration) &
    (detn['duration_days'] <= max_duration)
]

detn = detn_filtered.copy()
logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')


DEBUG:my_logger:584,(10322, 2333)
DEBUG:my_logger:((1766, 2335), np.int64(584)),0.33069082672706684


In [50]:

detn = reduce_dimensionality(detn,['household_adults','household_slept','living_children'],'household_adults_slept_living_children_z')

detn = reduce_dimensionality(detn,['weekly_avg_muac','weekly_last_wfh',],'muac_wfh')

detn['muac_diff_ratio'] = -detn['muac_diff_ratio']
detn['muac_diff_ratio_rate'] = -detn['muac_diff_ratio_rate']


detn = reduce_dimensionality(detn,['muac_diff_ratio','muac','muac_diff_ratio_rate'],'muac_diff_ratio_rate_z')




detn = reduce_dimensionality(detn,['duration_days','wk1_calc_los'],'duration_z')
#detn.drop(columns=['duration_days','wk1_calc_los'],inplace=True)
logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')

DEBUG:my_logger:((1766, 2339), np.int64(584)),0.33069082672706684


## predictive model feature importances

### autogluon

In [51]:
#explainer,predictor,ag_features,y_pred,y_pred_proba,ag_model_f1,X_test_ag,y_test_ag,y_pred_proba_all = ag_model_load(label,.2,detn)

y_pred_proba_all,explainer,ag_features = run_ag_model(label,detn,'')

DEBUG:my_logger:ag features:, 5, ['duration_z', 'wk1_weight_diff_rate', 'wk1_muac', 'muac_diff_ratio_rate_z', 'hl_trend']


In [52]:
pred_proba_all_series = y_pred_proba_all[1].rename(f'probability_{label}')
pid_probabilities = pid_probabilities.join(pred_proba_all_series)
logger.debug(pid_probabilities.shape)

DEBUG:my_logger:(10322, 24)


In [53]:
pid_probabilities[f'percentrank_{label}'] = pid_probabilities[f'probability_{label}'].rank(pct=True)
top_pct_pids = pid_probabilities[pid_probabilities[f'percentrank_{label}'] > TOP_PCT]['pid'].unique()
if EXPORT_SHAP_WATERFALL:
  json_series = export_waterfall_shap_values(explainer,detn[(detn['pid'].isin(active_pids)) & (detn['pid'].isin(top_pct_pids)) & (detn[label]== 0)],ag_features)
  pid_probabilities = pd.merge(pid_probabilities, json_series.rename(f'{label}_shap_data'), left_on='pid', right_index=True, how='left')


  0%|          | 0/12 [00:00<?, ?it/s]

### survival curves

# death

In [54]:
label = 'status_dead'


#detn = read_detn(label)
with open(dir + f'{label}.pkl', 'rb') as f:
  detn = pickle.load(f)

logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
# (8352, 1844) 233 0.02789750957854406

DEBUG:my_logger:((10322, 2331), np.int64(253)),0.024510753729897305


In [55]:
# TODO move this to etl_deterioration
detn = pd.merge(detn, admit_current[['pid','status','status_date']], on='pid', how='inner')
logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')
# prompt: create column in detn called final_date which is status_date or wk1_calcdate_weekly whichever is greater

# Create 'final_date' column, choosing the later date between 'status_date' and 'wk1_calcdate_weekly'
detn['final_date'] = detn[['status_date', 'wk1_calcdate_weekly']].max(axis=1)

# Fill NaN (active patients w/o a visit) values in 'final_date' with today's date
detn['final_date'] = detn['final_date'].fillna(datetime.date.today())
detn['final_date'] = pd.to_datetime(detn['final_date'])
detn['duration_days'] = detn['final_date'] - detn['calcdate']
detn['duration_days'] = detn['duration_days'].dt.days

#detn['wk1_calc_los'].fillna(detn['duration_days']/7,inplace=True)



DEBUG:my_logger:((10322, 2333), np.int64(253)),0.024510753729897305


In [56]:

label_date = f'{label}_date'
# for the majority that are missing label_date
detn['duration_days'] = detn['calcdate'].max() - detn['calcdate']

# overwrite only for those with non-null label_date
detn.loc[detn[label_date].notnull(), 'duration_days'] = detn.loc[detn[label_date].notnull(), label_date] - detn.loc[detn[label_date].notnull(), 'calcdate']

detn['duration_days'] = detn['duration_days'].dt.days

In [57]:
detn['wk1_calc_los'].fillna(0,inplace=True)

LOS_CUTOFF = 11
MUAC_CUTOFF = 12.1
NULL_MUAC_LOS_CUTOFF = 4
DURATION_DAYS_CUTOFF = 101

logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')

detn = detn[(((detn['weekly_last_muac'].isnull()) & (detn['wk1_calc_los'] < NULL_MUAC_LOS_CUTOFF)) & (detn['duration_days'] < DURATION_DAYS_CUTOFF) | ((detn['weekly_last_muac'] < MUAC_CUTOFF) & (detn['wk1_calc_los'] < LOS_CUTOFF)))]

logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')


DEBUG:my_logger:((10322, 2335), np.int64(253)),0.024510753729897305
DEBUG:my_logger:((1503, 2335), np.int64(242)),0.16101131071190952


In [58]:

detn = reduce_dimensionality(detn,['household_adults','household_slept','living_children'],'household_adults_slept_living_children_z')

detn = reduce_dimensionality(detn,['resp_rate', 'temperature'],'resp_rate_temperature_z')


detn = reduce_dimensionality(detn,['weekly_avg_muac','weekly_last_wfh',],'muac_wfh')

detn['muac_diff_ratio'] = -detn['muac_diff_ratio']

detn = reduce_dimensionality(detn,['muac_diff_ratio','muac'],'muac_diff_ratio_z')


logger.debug(f'{detn.shape,detn[label].sum()},{detn[label].mean()}')

detn = reduce_dimensionality(detn,['wfa_trend','hfa_trend'],'wfa_hfa_trend_z')

detn = reduce_dimensionality(detn,['cat1_complications_weekly','admit_cat1_complications'],'cat1_complications_z')

detn = reduce_dimensionality(detn,['wk1_rainy_season_weekly','lean_season_admit'],'season_z')

detn = reduce_dimensionality(detn,['wfh_rsquared','wfh_trend'],'wfh_trend_z')



DEBUG:my_logger:((1503, 2339), np.int64(242)),0.16101131071190952


In [59]:
# prompt: clip lower boundary of detn['duration_days'] to 0

detn['duration_days'] = detn['duration_days'].clip(lower=0)


In [60]:
detn_admit_only,_,_,_ = split_detn_new_onset_medical_complication(detn,label)
logger.debug(f'{detn_admit_only.shape,detn_admit_only[label].sum()},{detn_admit_only[label].mean()}')

DEBUG:my_logger:((383, 508), np.int64(154)),0.402088772845953


(383, 508) (219, 1449) (213, 1896) (688, 2343)


In [61]:
# Find 'pid' values in detn that are NOT in detn_admit_only
pid_not_in_admit = detn[~detn['pid'].isin(detn_admit_only['pid'])]['pid']

# Get rows from detn where 'pid' is in pid_not_in_admit
detn_filtered = detn[detn['pid'].isin(pid_not_in_admit)].copy()
detn_filtered.replace([np.inf, -np.inf], np.nan, inplace=True)



## predictive models

### autogluon

#### stratified models by length of service

In [62]:
y_pred_proba_all1,explainer1,ag_features1 = run_ag_model(label,detn_admit_only,'1')
y_pred_proba_all2,explainer2,ag_features2 = run_ag_model(label,detn_filtered,'not1')


Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.12, current=3.11.13)
DEBUG:my_logger:ag features:, 12, ['household_adults_slept_living_children_z', 'age_takefamily', 'gombe_lga', 'b_knowsbday', 'c_admission', 'estdist_home', 'resp_rate', 'duration_days', 'temperature', 'lean_season_admit', 'admit_cat1_complications', 'rainy_season_admit']
Found 1 mismatches between original and current metadata:
	INFO: AutoGluon Python micro version mismatch (original=3.11.12, current=3.11.13)
DEBUG:my_logger:ag features:, 10, ['duration_days', 'season_z', 'wk2_muac', 'wk1_wfhz', 'cg_weight', 'wfa_rsquared', 'weekly_first_muac', 'wfh_trend_z', 'cat1_complications_z', 'wk2_wfa_diff_rate']


In [63]:

y_pred_proba_all_stratified = pd.concat([y_pred_proba_all1,y_pred_proba_all2],axis=0)
y_pred_proba_all_stratified_series = y_pred_proba_all_stratified[1].rename(f'probability_{label}_stratified')
pid_probabilities = pid_probabilities.join(y_pred_proba_all_stratified_series)
logger.debug(pid_probabilities.shape)

DEBUG:my_logger:(10322, 27)


In [64]:
pid_probabilities[f'percentrank_{label}_stratified'] = pid_probabilities[f'probability_{label}_stratified'].rank(pct=True)
top_pct_pids = pid_probabilities[pid_probabilities[f'percentrank_{label}_stratified'] > TOP_PCT]['pid'].unique()


In [65]:
pid_probabilities[f'percentrank_{label}_stratified'] = pid_probabilities[f'probability_{label}_stratified'].rank(pct=True)
top_pct_pids = pid_probabilities[pid_probabilities[f'percentrank_{label}_stratified'] > TOP_PCT]['pid'].unique()
if EXPORT_SHAP_WATERFALL:
  json_series = export_waterfall_shap_values(explainer1,detn_admit_only[(detn_admit_only['pid'].isin(active_pids)) & (detn_admit_only['pid'].isin(top_pct_pids)) & (detn_admit_only[label]== 0)],ag_features1)
  json_series2 = export_waterfall_shap_values(explainer2,detn_filtered[(detn_filtered['pid'].isin(active_pids)) & (detn_filtered['pid'].isin(top_pct_pids)) & (detn_filtered[label]== 0)],ag_features2)
  pid_probabilities = pd.merge(pid_probabilities, pd.concat([json_series, json_series2]).rename(f'{label}_shap_data'), left_on='pid', right_index=True, how='left')


  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/179 [00:00<?, ?it/s]

In [66]:
# clip for better visualization and so AFT models will work, guaranteeing all durations >0
# doesn't affect models
#max_duration = detn[detn[label]==1]['duration_days'].max() +1
#print(max_duration)

# prompt: clip detn['duration_days'] to max_duration for K-M clarity

# detn['duration_days'] = detn['duration_days'].clip(upper=max_duration, lower=.01)

detn['duration_days'] = detn['duration_days'].clip(lower=.01)

### Survival Curves

### AFT survival regression

In [67]:
#regr_cols = ['wk1_muac','wk1_calc_los','muac',label,'duration_days','pid']
regr_cols = ['wk1_muac','muac',label,'duration_days','pid']
regression_dataset = detn[regr_cols].copy()
regression_dataset['wk1_muac'].fillna(regression_dataset['muac'],inplace=True)
regression_dataset.drop(columns=['muac'],inplace=True)

regression_dataset.dropna(inplace=True)


#regression_dataset['duration_days'] = regression_dataset['duration_days'].clip(lower=.01)
aft = WeibullAFTFitter()

aft.fit(regression_dataset, duration_col='duration_days', event_col=f'{label}', ancillary=True, formula=' wk1_muac')

logger.debug(aft.print_summary())

<lifelines.WeibullAFTFitter: fitted with 1451 total observations, 1231 right-censored observations>
             duration col = 'duration_days'
                event col = 'status_dead'
   number of observations = 1451
number of events observed = 220
           log-likelihood = -1352.47
         time fit was run = 2025-06-14 23:06:10 UTC

---
                   coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                  
lambda_ Intercept -5.08      0.01      1.47           -7.97           -2.19                0.00                0.11
        wk1_muac   1.17      3.21      0.15            0.87            1.46                2.38                4.32
rho_    Intercept -1.48      0.23      0.34           -2.15           -0.81                0.12                0.44
        wk1_muac   0.08      1.08      0.03            0.01            0.14                1.01                1.15

                   cmp to     z      p  -log2(p)
param   covariate                               
lambda_ Intercept    0.00 -3.44 <0.005     10.76
        wk1_muac     0.00  7.67 <0.005     45.72
rho_    Intercept    0.00 -4.33 <0.005     16.06
        wk1_muac     0.00  2.27   0.02      5.41
---
Concordance = 0.75
AIC = 2712.95
log-likelihood ratio test = 145.77 on 2 df
-log2(p) of ll-ratio test = 105.15

DEBUG:my_logger:None


### censored (i.e., still alive) patients prediction

In [68]:
regr_cols = ['wk1_muac','muac',label,'duration_days','pid']
censored_subjects = detn[regr_cols].copy()
censored_subjects['wk1_muac'].fillna(censored_subjects['muac'],inplace=True)
censored_subjects.drop(columns=['muac'],inplace=True)


censored_subjects.dropna(inplace=True)

censored_subjects = censored_subjects.loc[detn[label]==0].copy()

censored_subjects_last_obs = censored_subjects['duration_days']
censored_subjects_last_obs = censored_subjects_last_obs.clip(lower=0.01)


In [69]:
y_pred_median = aft.predict_median(censored_subjects,conditional_after=censored_subjects_last_obs)
y_pred_median.rename('median_predicted_days_to_death',inplace=True)

duration_days_col = f'duration_days_{label}'

censored_subjects.rename(columns={'duration_days':duration_days_col},inplace=True)


df = pd.merge(censored_subjects[['pid',duration_days_col]],y_pred_median,left_index=True,right_index=True)

pid_probabilities =pd.merge(pid_probabilities,df,on='pid',how='left')
logger.debug(pid_probabilities.shape)




DEBUG:my_logger:(10322, 31)


In [71]:
# TODO write pid_probabilities to Postgres table
#pid_probabilities.to_excel('pid_probabilities9.xlsx',index=None)